In [1]:
import os
import mlflow
import numpy as np
import pandas as pd

from math import sqrt
from joblib import delayed
from joblib import Parallel

from warnings import catch_warnings
from warnings import filterwarnings
from multiprocessing import cpu_count
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from pathlib import Path
from functools import reduce
from datetime import datetime

from prophet import Prophet

Importing plotly failed. Interactive plots will not work.


# Data

In [14]:
df = pd.read_csv('./../../final_dfs/for_modeling/df_final_date_wide_2007.csv', converters={'ags2': str, 'ags5': str} )
df.set_index('ags5', drop=True, inplace=True)
#df = df.drop("Unnamed: 0", axis=1)
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
df = df[ (df['date'] < '2021-05-01')]
df = df.reset_index()
df

,ags5,date,number_of_company_deletions,number_of_company_liquidations,number_of_start_ups,number_of_companies_administration,number_of_companies_agriculture,number_of_companies_arts_entertainment,number_of_companies_communication,number_of_companies_construction,...,employees_social_security_at_residence,employees_social_security_at_work,realized_short_time_work_companies,realized_short_time_work_people,registerd_jobs,underemployment_without_short_time _work,unemployed,unemployment_benefit_entitled,unemployment_benefit_recipients,unemployment_rate
0,1001,2007-01-01,4.0,2.0,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1001,2007-02-01,5.0,3.0,12.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1001,2007-03-01,7.0,6.0,19.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1001,2007-04-01,5.0,1.0,11.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1001,2007-05-01,4.0,2.0,21.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68967,16077,2020-12-01,2.0,1.0,6.0,115.0,99.0,158.0,44.0,457.0,...,NaN,NaN,NaN,NaN,702.0,4147.0,3015.0,6196.000000,979.0,6.7
68968,16077,2021-01-01,0.0,3.0,2.0,115.0,99.0,158.0,44.0,459.0,...,NaN,NaN,NaN,NaN,681.0,NaN,3287.0,6236.000000,NaN,7.3
68969,16077,2021-02-01,0.0,0.0,5.0,115.0,99.0,158.0,43.0,457.0,...,NaN,NaN,NaN,NaN,680.0,NaN,3270.0,6248.782315,NaN,7.3
68970,16077,2021-03-01,4.0,0.0,4.0,114.0,100.0,158.0,42.0,455.0,...,NaN,NaN,NaN,NaN,749.0,NaN,3222.0,6219.427429,NaN,7.2


In [13]:
df = pd.read_csv('./../../final_dfs/for_modeling/df_final_stationery.csv', converters={'ags2': str, 'ags5': str} )
df.set_index('ags5', drop=True, inplace=True)
df = df.drop(['ags2','kreis','cluster'], axis=1)
#df = df[(df['date'] >= '2010-01-01') & (df['date'] < '2020-01-01')]
df = df.reset_index()
df

,ags5,supermarkets_population,supermarkets_average_distance,public_transport_availability,average_distance_bus_stop,average_distance_train_station,average_distance_public_transport,Unnamed:_0,number_of_students,number_of_hospitals,...,room_type_location,district_settlement_structure,type_of_settlement_structure,urban_/_rural,metropolitan_region,metropolitan_area,east_west,border_proximity,support_area_status,eligible_area
0,01001,92,500,35,240,2901,240,0,11610,3,...,2,4,3,2,99,99,1,1,C,1
1,01002,92,460,37,268,2037,265,1,23898,16,...,2,1,2,1,99,99,1,0,C/D,1
2,01003,90,532,37,297,1927,294,2,20478,13,...,1,1,2,1,5,99,1,0,C/D,1
3,01004,85,588,37,316,1648,313,3,10380,5,...,2,3,2,2,5,99,1,0,D,1
4,01051,51,1864,35,448,3517,443,4,13158,3,...,4,4,3,2,5,99,1,0,C,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,16073,62,1423,37,341,2492,322,396,8772,3,...,3,4,2,2,99,99,2,0,C*,1
397,16074,54,1677,38,343,3419,340,397,7190,3,...,3,3,2,2,99,99,2,0,C*,1
398,16075,54,1686,35,449,3172,444,398,7441,1,...,3,4,2,2,99,99,2,2,C*,1
399,16076,55,1654,38,608,2779,563,399,8403,2,...,3,2,2,1,99,99,2,0,C*,1


In [3]:
# select only column for which all values are avaliable
df = df[['date', 
        'number_of_company_deletions', 
        'number_of_company_liquidations', 
        'number_of_start_ups',
        'unemployment_rate', 
        'registerd_jobs',
        'employees_social_security_at_work',
        'employees_social_security_at_residence']]

In [4]:
# rename date and unemployment_rate for Prophet function 
df = df.rename(columns={'date': 'ds', 'unemployment_rate': 'y'})

# 401 Model for each kreis

In [15]:
# one-step forecast
def Prophet_forecast(data, config):
    g, sps, ys, sm, nc = config
    # define model
    model = Prophet(growth = g, seasonality_prior_scale = sps, yearly_seasonality = ys, seasonality_mode = sm, 
                   n_changepoints= nc)
    # fit model
    model_fit = model.fit(data)
    # make one step forecast
    future = model_fit.make_future_dataframe(periods=12, freq='MS')
    res = model_fit.predict(future)
    res.to_csv(f"runs_res/{g}_{sps}_{ys}_{sm}")
    return res.iloc[-12:][["ds","yhat"]]

In [16]:
df_ags5 = df.filter(regex='1001', axis=0)
config = ['linear', 15.0, True, 'additive', 80]
Prophet_forecast(df_ags5, config)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,yhat
120,2020-01-01,8.187907
121,2020-02-01,8.133685
122,2020-03-01,7.941841
123,2020-04-01,7.902734
124,2020-05-01,7.319465
125,2020-06-01,7.312372
126,2020-07-01,7.613619
127,2020-08-01,7.578347
128,2020-09-01,7.232740
129,2020-10-01,7.094384


In [17]:
# root mean squared error or rmse
def measure_rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))

In [18]:
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
    return data[:-n_test], data[-n_test:]

In [19]:
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
    # split dataset
    train, test = train_test_split(data, n_test)
    # fit model and make forecast for history
    yhat = Prophet_forecast(train, cfg)
    # estimate prediction error
    error = measure_rmse(test['y'], yhat['yhat'])
    return error

In [20]:
def run_model(data, n_test): 
        for g in ['linear']: 
            for sps in [10.0]: 
                for ys in [True]: 
                    for sm in ['multiplicative']:
                        for nc in [50, 85, 95]:
                            with mlflow.start_run():
                                params = [g, sps, ys, sm, nc]

                                ags5 = list(set(df.index.values.tolist()))
                                rmse = []
                                for i in ags5:
                                    df_ags5 = df.filter(regex=i, axis=0)
                                    current_rmse = walk_forward_validation(df_ags5, n_test, params)
                                    print(current_rmse)
                                    rmse.append(current_rmse)

                                # Log params
                                params = {"growth":g, "sps":sps, "ys":ys, "sm":sm, "nc":nc}

                                mlflow.log_params(params)

                                # Log metrics
                                mlflow.log_metric('rmse', np.mean(rmse))
                                mlflow.log_metric('mse', np.mean(rmse)**2)

In [ ]:
run_model(df, 12)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


0.25261898534665056


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


0.39339192427991193


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 85.


0.12681286031154204


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 85.


0.4065489779926285


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [20]:
np.mean([10.0, 15.0])

12.5

In [9]:
# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
    result = None
    # convert config to a key
    key = str(cfg)
    # show all warnings and fail on exception if debugging
    if debug:
        result = walk_forward_validation(data, n_test, cfg)
    else:
            # one failure during model validation suggests an unstable config
        try:
            # never show warnings when grid searching, too noisy
            with catch_warnings():
                filterwarnings("ignore")
                result = walk_forward_validation(data, n_test, cfg)
        except:
            error = None
    # check for an interesting result
#     if result is not None:
#         print(' > Model[%s] %.3f' % (key, result))
        
    print('.', end='')
    return (key, result)

----------
!mlflow experiments create --experiment-name 401ModelsTemp

Created experiment '401ModelsTemp' with id 6

---------

In [ ]:
# data split
n_test = 12
# model config
cfg = [('linear'), (10.0)]
# train 401 models
print("Training", end=' ')
failed_ags5 = []
rmse_scores = []

with mlflow.start_run(experiment_id=6):
    
    for ags5, df_for_kreis in df_y_grouped:
        
        data = list(df_for_kreis.sort_values('ds').drop(['ds'], axis=1).copy()['y'])
        
        try: 
        
            score = score_model(data, n_test, cfg)

            if score is None:
                raise Exception("No Score")
            
            rmse_scores.append(score[1])
            print('.', end='')
        except:
            failed_ags5.append(ags5)
            print('!', end='')
            
    params = {}
    params['p'], params['d'], params['q'] = cfg[0]
    params['s_p'], params['s_d'], params['s_q'], params['m'] = cfg[1]
    params['t'] = cfg[2]
    
    net_rmse = np.mean([rmse**2 for rmse in rmse_scores])**(0.5)
    
    
    mlflow.log_params(params)
    mlflow.log_metric('net_rmse', net_rmse)
      
    

print('\ndone')
print("Failed ags5:", failed_ags5)

Training ........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [ ]:
rmse_scores

In [ ]:
params

In [37]:
m = Prophet()
m.fit(df)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [38]:
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
480,2020-11-26
481,2020-11-27
482,2020-11-28
483,2020-11-29
484,2020-11-30


In [2]:
Prophet?